In [160]:
import pandas as pd
import numpy as np
import warnings

warnings.simplefilter('ignore')

raw_prices = pd.read_csv(r'C:\Users\Andre\OneDrive\5 - André - BI\Utilitários\Raw_prices.csv')
negocios = pd.read_csv(r'C:\Users\Andre\OneDrive\5 - André - BI\Utilitários\Negocios.csv')
financeiro = pd.read_csv(r'C:\Users\Andre\OneDrive\5 - André - BI\Utilitários\Financeiro.csv')

In [81]:
raw_prices['Datas'] = pd.to_datetime(raw_prices['Datas'])

In [82]:
raw_prices.head()

,Datas,RRRP3,TTEN3,QVUM3B,QVQP3B,APPA3,APPA4,ABCB3,ABCB4,ABCB11,...,WLMM4,WWOW3,ILMD3,ILMD4,YBRA3B,YDUQ3,ESTC4,ESTC11,OPZI3B,ZIVI4
0,1986-01-02,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1986-01-03,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,2.862124,NaN,NaN,98.087575,NaN,NaN,NaN,NaN,NaN,NaN
2,1986-01-06,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,96.125827,NaN,NaN,NaN,NaN,NaN,NaN
3,1986-01-07,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,3.892488,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1986-01-08,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,4.350428,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [83]:
raw_prices.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9303 entries, 0 to 9302
Columns: 1383 entries, Datas to ZIVI4
dtypes: datetime64[ns](1), float64(1382)
memory usage: 98.2 MB


In [94]:
precos_corte = raw_prices[(raw_prices['Datas'] >= '2000-01-01') & (raw_prices['Datas'] <= '2020-12-31')]

In [95]:
long_data = precos_corte.melt(id_vars=['Datas'], var_name='Codigo', value_name='Preco')

In [97]:
long_data['Ano'] = long_data['Datas'].dt.year

In [87]:
grouped_data = long_data.groupby(['Codigo', 'Ano'])

In [98]:
long_data['Total_Dias_Negociacao'] = long_data.groupby(['Codigo', 'Ano'])['Datas'].transform('nunique')
long_data['Pregoes'] = long_data.groupby(['Codigo', 'Ano'])['Preco'].transform(lambda x: x.notna().sum())
long_data['Pregoes_Percentual'] = long_data['Pregoes'] / long_data['Total_Dias_Negociacao']


In [101]:
long_data = long_data[long_data['Pregoes_Percentual'] > 0.9]

In [104]:
long_data = long_data.reset_index(drop=True)

In [107]:
long_data['Preco'] = long_data['Preco'].fillna(method='ffill')
long_data['Retorno'] = long_data.groupby('Codigo')['Preco'].pct_change()
long_data['Risco'] = long_data.groupby(['Codigo', 'Ano'])['Retorno'].transform(lambda x: np.sqrt(len(x)) * np.nanstd(x))

In [116]:
long_data['Retorno_positivos'] = long_data.groupby(['Codigo', 'Ano'])['Retorno'].transform(lambda x: (x > 0).sum())
long_data['Retorno_negativos'] = long_data.groupby(['Codigo', 'Ano'])['Retorno'].transform(lambda x: (x < 0).sum())
long_data['Perc_positivos'] = long_data['Retorno_positivos'] / (long_data['Retorno_positivos'] + long_data['Retorno_negativos'])
long_data['Perc_negativos'] = long_data['Retorno_negativos'] / (long_data['Retorno_positivos'] + long_data['Retorno_negativos'])
long_data['Retorno'] = long_data['Retorno'].fillna(0)
long_data['Retorno_Anual'] = long_data.groupby(['Codigo', 'Ano'])['Retorno'].transform(lambda x: np.prod(1 + x) - 1)
long_data['Retorno_acumulado'] = long_data.groupby('Codigo')['Retorno'].transform(lambda x: (1 + x).cumprod() - 1)
long_data['FIP'] = long_data['Perc_negativos'] - long_data['Perc_positivos']

In [132]:
dados_agregados = long_data.groupby(['Codigo', 'Ano']).agg({'Retorno_Anual':'first', 'Risco':'first', 'FIP':'first'}).reset_index()

In [134]:
dados_agregados['Decil_Retorno'] = dados_agregados.groupby('Ano')['Retorno_Anual'].transform(lambda x: pd.qcut(x, 10, labels=False, duplicates='drop') + 1)
dados_agregados['Decil_Risco'] = dados_agregados.groupby('Ano')['Risco'].transform(lambda x: pd.qcut(x, 10, labels=False, duplicates='drop') + 1)
dados_agregados['Decil_FIP'] = dados_agregados.groupby(['Codigo', 'Ano'])['FIP'].transform(lambda x: pd.qcut(x, 10, labels=False, duplicates='drop') + 1)

In [138]:
dados_agregados['Decil_Retorno'] = dados_agregados.groupby('Ano')['Retorno_Anual'].transform(lambda x: pd.qcut(x, 10, labels=False, duplicates='drop') + 1)
dados_agregados['Decil_Risco'] = dados_agregados.groupby('Ano')['Risco'].transform(lambda x: pd.qcut(x, 10, labels=False, duplicates='drop') + 1)
dados_agregados['Decil_FIP'] = dados_agregados.groupby('Ano')['FIP'].transform(lambda x: pd.qcut(x, 10, labels=False, duplicates='drop') + 1)


In [140]:
long_data = pd.merge(long_data, dados_agregados[['Codigo', 'Ano', 'Decil_Retorno', 'Decil_Risco', 'Decil_FIP']], on=['Codigo', 'Ano'], how='left')

In [143]:
long_data.sample(50)

,Datas,Codigo,Preco,Ano,Total_Dias_Negociacao,Pregoes,Pregoes_Percentual,Retorno,Risco,Retorno_positivos,Retorno_negativos,Perc_positivos,Perc_negativos,Retorno_Anual,Retorno_acumulado,FIP,Decil_Retorno,Decil_Risco,Decil_FIP
759412,2008-02-22,RDNI3,36.929508,2008,262,249,0.950382,0.022410,2121.024877,89,137,0.393805,0.606195,745.287132,2417.732785,0.212389,10,10,9
778373,2020-10-19,SAPR3,5.333331,2020,262,249,0.950382,-0.011561,0.602996,122,117,0.510460,0.489540,0.021033,-0.293735,-0.020921,6,4,3
741775,2005-07-27,RAPT4,4.701638,2005,260,249,0.957692,0.034545,0.260678,96,124,0.436364,0.563636,0.016676,-0.599884,0.127273,4,1,9
1043024,2011-01-06,YDUQ3,11.713702,2011,260,249,0.957692,-0.005981,0.354488,105,132,0.443038,0.556962,-0.361638,-0.660236,0.113924,3,5,9
717387,2019-02-12,PSSA3,62.717724,2019,261,248,0.950192,0.047397,0.245385,135,108,0.555556,0.444444,0.199648,7.044462,-0.111111,4,2,1
739933,2020-07-08,RADL3,24.493962,2020,262,249,0.950382,-0.002217,0.410147,123,124,0.497976,0.502024,0.079084,-0.533101,0.004049,7,1,5
703805,2020-01-23,PRIO3,10.189527,2020,262,249,0.950382,0.007143,1.091411,118,130,0.475806,0.524194,1.031347,-0.829569,0.048387,10,9,7
167434,2010-07-29,BISA3,14.334264,2010,261,247,0.946360,0.000000,0.407611,117,120,0.493671,0.506329,0.062131,-0.919531,0.012658,5,7,5
349030,2019-07-03,ENEV3,6.965700,2019,261,248,0.950192,0.010944,0.256752,145,96,0.601660,0.398340,1.603254,-0.867925,-0.203320,10,2,1
946420,2006-10-26,TCSL4,13.425779,2006,260,246,0.946154,0.056489,0.425470,122,117,0.510460,0.489540,0.233344,-0.150280,-0.020921,4,7,4


In [161]:
negocios['Datas'] = pd.to_datetime(negocios['Datas'])
negocios = negocios[(negocios['Datas'] >= '2000-01-01') & (negocios['Datas'] <= '2020-12-31')]
negocios = negocios.melt(id_vars=['Datas'], var_name='Codigo', value_name='Negocios')
negocios['Negocios'] = negocios['Negocios'].fillna(0)
negocios['Ano'] = negocios['Datas'].dt.year
negocios['Media_Negocios_Acao'] = negocios.groupby(['Codigo', 'Ano'])['Negocios'].transform(lambda x: x.mean(skipna=True))
negocios['Dias_Negocios'] = negocios.groupby(['Codigo', 'Ano'])['Negocios'].transform(lambda x: x.notna().sum())